In [8]:
import pandas as pd
from omegaconf import OmegaConf
from lib.uncertainty import Simulate

In [10]:
EXPERIMENTS = [
    '2026-01-20_11-18'
]

strategy = {
    'oracle': 'Oracle',
    'random': 'Random',
    'similar': 'Similar',
    'dense': 'Dense'
}

In [3]:
test = EXPERIMENTS[0]

In [4]:
results_all = {}

for experiment in EXPERIMENTS:
    cfg = OmegaConf.load(f'../results/{experiment}/config.yaml')
    results = pd.read_json(f'../results/{experiment}/results.json')
    results['correct_query'] = True
    results['retriever_strategy'] = cfg.retriever.strategy

    results_all[cfg.retriever.strategy] = results

In [11]:
from prettytable import PrettyTable

simulate = Simulate(cfg)

t = PrettyTable(field_names=['Retriever Strategy', 'Query', 'Retreiver', 'Generator', 'Rows'])

for experiment, results in results_all.items():
    success_rates = simulate.compute_uncertainty(results)

    t.add_row([
        strategy[experiment],
        f"{success_rates['q']['mean']:.2%} ± {success_rates['q']['std']:.4%}",
        f"{success_rates['r']['mean']:.2%} ± {success_rates['r']['std']:.4%}",
        f"{success_rates['g']['mean']:.2%} ± {success_rates['g']['std']:.4%}",
        len(results)
    ])


t

Retriever Strategy,Query,Retreiver,Generator,Rows
Similar,91.67% ± 7.6878%,11.77% ± 8.3568%,77.92% ± 10.1035%,10
